In [1]:
import os

In [2]:
%pwd

'/Users/phuc.buidang/Documents/Learn/Projects/basics_ds_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/phuc.buidang/Documents/Learn/Projects/basics_ds_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

2025-04-06 11:03:00.130 | DEBUG    | ipykernel.kernelbase:execute_request:802 - {'header': {'msg_id': '72cbe922-8b95cd444f6bf161e7aacbe2_25286_55', 'msg_type': 'execute_reply', 'username': 'phuc.buidang', 'session': '72cbe922-8b95cd444f6bf161e7aacbe2', 'date': datetime.datetime(2025, 4, 6, 4, 3, 0, 130035, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '72cbe922-8b95cd444f6bf161e7aacbe2_25286_55', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2025, 4, 6, 4, 2, 59, 964000, tzinfo=tzutc()), 'msg_id': '94ed900e-4d7f-4b46-8b43-8717f93f6291', 'msg_type': 'execute_request', 'session': 'e1c3e43e-bb5c-4065-8ab2-b05f2093729e', 'username': '82bd650b-4e46-4cea-b744-3fce9086a129', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 6, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2025, 4, 6, 4, 2, 59, 965520, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': '232cdb24-9cb9-488e-99af-30d8d11e8

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )

        return model_trainer_config

2025-04-06 11:03:00.132 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 11:03:00.132 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'class ConfigurationManager:\n    def __init__(\n        self,\n        config_filepath=CONFIG_FILE_PATH,\n        params_filepath=PARAMS_FILE_PATH,\n        schema_filepath=SCHEMA_FILE_PATH,\n    ):\n\n        self.config = read_yaml(config_filepath)\n        self.params = read_yaml(params_filepath)\n        self.schema = read_yaml(schema_filepath)\n\n        create_directories([self.config.artifacts_root])\n\n    def get_model_trainer_config(self) -> ModelTrainerConfig:\n        config = self.config.model_trainer\n        params = self.params.ElasticNet\n        schema = self.schema.TARGET_COLUMN\n\n        create_directories([config.root_dir])\n\n        model_tra

In [8]:
import pandas as pd
import os
from sklearn.linear_model import ElasticNet
import joblib

2025-04-06 11:03:00.148 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 11:03:00.149 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'import pandas as pd\nimport os\nfrom sklearn.linear_model import ElasticNet\nimport joblib'}
   --->
   
2025-04-06 11:03:00.149 | DEBUG    | ipykernel.kernelbase:dispatch_shell:429 - execute_request: {'header': {'date': datetime.datetime(2025, 4, 6, 4, 3, 0, 148000, tzinfo=tzutc()), 'msg_id': 'd711ff6e-2990-47b2-98ee-8adfdb6ce8ae', 'msg_type': 'execute_request', 'session': 'e1c3e43e-bb5c-4065-8ab2-b05f2093729e', 'username': '82bd650b-4e46-4cea-b744-3fce9086a129', 'version': '5.2'}, 'msg_id': 'd711ff6e-2990-47b2-98ee-8adfdb6ce8ae', 'msg_type': 'execute_request', 'parent_header': {}, 'metadata': {'cellId': 'vscode-notebook-cell:/Users/phuc.buidang/Documents/Learn/Pro

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(
            alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42
        )
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

2025-04-06 11:03:01.280 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 11:03:01.281 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'class ModelTrainer:\n    def __init__(self, config: ModelTrainerConfig):\n        self.config = config\n\n    def train(self):\n        train_data = pd.read_csv(self.config.train_data_path)\n        test_data = pd.read_csv(self.config.test_data_path)\n\n        train_x = train_data.drop([self.config.target_column], axis=1)\n        test_x = test_data.drop([self.config.target_column], axis=1)\n        train_y = train_data[[self.config.target_column]]\n        test_y = test_data[[self.config.target_column]]\n\n        lr = ElasticNet(\n            alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42\n        )\n        lr.fit(train_x, train_y)\n\n  

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

2025-04-06 11:03:01.285 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 11:03:01.286 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'try:\n    config = ConfigurationManager()\n    model_trainer_config = config.get_model_trainer_config()\n    model_trainer = ModelTrainer(config=model_trainer_config)\n    model_trainer.train()\nexcept Exception as e:\n    raise e'}
   --->
   
2025-04-06 11:03:01.286 | DEBUG    | ipykernel.kernelbase:dispatch_shell:429 - execute_request: {'header': {'date': datetime.datetime(2025, 4, 6, 4, 3, 1, 285000, tzinfo=tzutc()), 'msg_id': 'ca4bc8da-8ab1-486c-b16c-deb6ee9f1201', 'msg_type': 'execute_request', 'session': 'e1c3e43e-bb5c-4065-8ab2-b05f2093729e', 'username': '82bd650b-4e46-4cea-b744-3fce9086a129', 'version': '5.2'}, 'msg_id': 'ca4bc8da-8ab1-486c-b16c-deb6ee9f120